In [1]:
from ctypes import *
dll = WinDLL("CTA-railway.dll")

In [2]:
dll.initSim()

376

In [3]:
dll.runSim()
dll.SimIsFinished()

0

In [4]:
dll.getTotalTravelTime.restype = c_double   
totalTravelTime = dll.getTotalTravelTime()/3600.0
print('%.1f' % totalTravelTime)

146088.9


In [5]:
dll.getTotalDelay.restype = c_double   
totalDelay = dll.getTotalDelay()/3600.0
print('%.1f' % totalDelay)

38139.7


In [ ]:
# reset测试
dll.resetSim()
dll.runSim()
dll.getTotalDelay.restype = c_double   
totalDelay = dll.getTotalDelay()/3600.0
print('%.1f' % totalDelay)

In [ ]:
# 效率测试
import time

time_start=time.time()
for i in range(10):
    dll.resetSim()
    dll.runSim()
    dll.getTotalDelay.restype = c_double   
    totalDelay = dll.getTotalDelay()/3600.0
time_end=time.time()
print('time test (s):\t',(time_end-time_start)/10)

In [6]:
# 分析各个站的残留人数
import numpy as np

dll.getStationWaitingPassengers.argtypes = [c_int, c_int]
dll.getStationWaitingPassengers.restype = c_int
count = 0
lefts = []
for station in range(252):
    buffer = [station]
    buffer.append(dll.getStationWaitingPassengers(station, 0))
    buffer.append(dll.getStationWaitingPassengers(station, 1))
    buffer.append(buffer[1]+buffer[2])
    lefts.append(buffer)
    count += dll.getStationWaitingPassengers(station, 0)
    count += dll.getStationWaitingPassengers(station, 1)
print(count, " passengers left in the stations after the Simulation")

left = np.array(lefts)
top_left = left[np.lexsort(-left.T)]
for i in range(252):
    print("Station ID: ", '%.0f'%top_left[i][0], "\tLeft passengers: ", '%d'%(top_left[i][3]))

190312  passengers left in the stations after the Simulation
Station ID:  194 	Left passengers:  9039
Station ID:  230 	Left passengers:  8861
Station ID:  231 	Left passengers:  7480
Station ID:  192 	Left passengers:  6877
Station ID:  96 	Left passengers:  5914
Station ID:  196 	Left passengers:  5624
Station ID:  115 	Left passengers:  5510
Station ID:  159 	Left passengers:  5443
Station ID:  223 	Left passengers:  5328
Station ID:  235 	Left passengers:  4721
Station ID:  146 	Left passengers:  4378
Station ID:  227 	Left passengers:  4095
Station ID:  158 	Left passengers:  3806
Station ID:  121 	Left passengers:  3663
Station ID:  124 	Left passengers:  3601
Station ID:  138 	Left passengers:  3495
Station ID:  241 	Left passengers:  3423
Station ID:  210 	Left passengers:  3376
Station ID:  222 	Left passengers:  3110
Station ID:  127 	Left passengers:  3094
Station ID:  168 	Left passengers:  2857
Station ID:  104 	Left passengers:  2739
Station ID:  103 	Left passengers:  26

In [7]:
# 分析各个站的delay
import numpy as np

dll.getStationDelay.argtypes = [c_int, c_int]
dll.getStationDelay.restype = c_double
delays = []
for station in range(252):
    buffer = [station]
    buffer.append(dll.getStationDelay(station, 0))
    buffer.append(dll.getStationDelay(station, 1))
    buffer.append(buffer[1]+buffer[2])
    delays.append(buffer)

delay = np.array(delays)
top_delay = delay[np.lexsort(-delay.T)]
for i in range(252):
    print("Station ID: ", '%.0f'%top_delay[i][0], "\tDelay (h): ", '%.1f'%(top_delay[i][3]/3600))

Station ID:  190 	Delay (h):  9654.3
Station ID:  4 	Delay (h):  2609.3
Station ID:  92 	Delay (h):  2503.3
Station ID:  15 	Delay (h):  1241.0
Station ID:  25 	Delay (h):  804.3
Station ID:  52 	Delay (h):  706.1
Station ID:  85 	Delay (h):  700.2
Station ID:  122 	Delay (h):  623.1
Station ID:  23 	Delay (h):  608.2
Station ID:  24 	Delay (h):  573.6
Station ID:  28 	Delay (h):  538.5
Station ID:  71 	Delay (h):  523.6
Station ID:  89 	Delay (h):  493.8
Station ID:  22 	Delay (h):  472.7
Station ID:  35 	Delay (h):  408.5
Station ID:  100 	Delay (h):  398.7
Station ID:  182 	Delay (h):  394.5
Station ID:  78 	Delay (h):  361.4
Station ID:  76 	Delay (h):  350.0
Station ID:  61 	Delay (h):  346.0
Station ID:  13 	Delay (h):  318.7
Station ID:  33 	Delay (h):  307.8
Station ID:  39 	Delay (h):  296.8
Station ID:  37 	Delay (h):  286.5
Station ID:  1 	Delay (h):  285.0
Station ID:  20 	Delay (h):  283.5
Station ID:  50 	Delay (h):  283.2
Station ID:  87 	Delay (h):  277.5
Station ID:  6

In [8]:
# 分析各个站的人均delay
import numpy as np

dll.getStationPass.argtypes = [c_int, c_int]
dll.getStationPass.restype = c_int
avg_delays = []
for station in range(252):
    buffer = [station]
    if dll.getStationPass(station, 0) != 0:
        buffer.append(dll.getStationDelay(station, 0)/float(dll.getStationPass(station, 0)))
    else:
        buffer.append(0)
    if dll.getStationPass(station, 1) != 0:
        buffer.append(dll.getStationDelay(station, 1)/float(dll.getStationPass(station, 1)))
    else:
        buffer.append(0)
    buffer.append(max(buffer[1],buffer[2]))
#     buffer.append(dll.getStationPass(station, 0))
#     buffer.append(dll.getStationPass(station, 1))
#     buffer.append(buffer[1]+buffer[2])
    avg_delays.append(buffer)

avg_delay = np.array(avg_delays)
top_avg_delay = avg_delay[np.lexsort(-avg_delay.T)]
for i in range(252):
    print("Station ID:  ", '%.0f'%top_avg_delay[i][0], "\tAvg_Delay0 (s): ", '%.1f\t'%(top_avg_delay[i][1]), 
          "\tAvg_Delay1 (s): ", '%.1f'%(top_avg_delay[i][2]))

Station ID:   190 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  13748.2
Station ID:   182 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  12795.4
Station ID:   122 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  11683.1
Station ID:   224 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  11590.9
Station ID:   132 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  10347.4
Station ID:   142 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  9458.1
Station ID:   162 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  9205.6
Station ID:   100 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  8493.0
Station ID:   66 	Avg_Delay0 (s):  658.1	 	Avg_Delay1 (s):  422.7
Station ID:   44 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  584.3
Station ID:   56 	Avg_Delay0 (s):  554.3	 	Avg_Delay1 (s):  528.7
Station ID:   17 	Avg_Delay0 (s):  0.0	 	Avg_Delay1 (s):  548.4
Station ID:   32 	Avg_Delay0 (s):  320.0	 	Avg_Delay1 (s):  248.6
Station ID:   63 	Avg_Delay0 (s):  312.1	 	Avg_Delay1 (s):  261.1
Station ID:   2 	Avg_Delay0 (s):  292.3	 	Avg_Delay1 (s):  306.7
Station ID